In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("dataset(calls).csv", skiprows=0, skipcolumn, header=0)

In [10]:
def prepare_data(file_path):
    # Read CSV while preserving original structure
    df = pd.read_csv(file_path, skiprows=0, header=0)
    
    # Clean column names
    df.columns = [col.strip() for col in df.columns]
    
    # Extract strike prices
    df["STRIKE"] = df["STRIKE"].apply(
        lambda x: float(str(x).replace(',', '')) if str(x).strip() not in ['-', ''] else np.nan
    )
    
    # Remove rows with invalid strike prices
    df = df.dropna(subset=['STRIKE'])
    
    # Create empty DataFrames for calls and puts
    calls = pd.DataFrame()
    puts = pd.DataFrame()
    
    # Manually map columns based on known structure
    # Calls data: columns 0-9
    calls['OI'] = df.iloc[:, 1]
    calls['CHNG_IN_OI'] = df.iloc[:, 2]
    calls['VOLUME'] = df.iloc[:, 3]
    calls['IV'] = df.iloc[:, 4]
    calls['LTP'] = df.iloc[:, 5]
    calls['CHNG'] = df.iloc[:, 6]
    calls['BID_QTY'] = df.iloc[:, 7]
    calls['BID'] = df.iloc[:, 8]
    calls['ASK'] = df.iloc[:, 9]
    calls['ASK_QTY'] = df.iloc[:, 10]
    calls['Strike Price'] = df['STRIKE']
    calls['Option Type'] = 'call'
    
    # Puts data: columns 11-20
    puts['BID_QTY'] = df.iloc[:, 12]
    puts['BID'] = df.iloc[:, 13]
    puts['ASK'] = df.iloc[:, 14]
    puts['ASK_QTY'] = df.iloc[:, 15]
    puts['CHNG'] = df.iloc[:, 16]
    puts['LTP'] = df.iloc[:, 17]
    puts['IV'] = df.iloc[:, 18]
    puts['VOLUME'] = df.iloc[:, 19]
    puts['CHNG_IN_OI'] = df.iloc[:, 20]
    puts['OI'] = df.iloc[:, 21]
    puts['Strike Price'] = df['STRIKE']
    puts['Option Type'] = 'put'
    
    # Combine calls and puts
    options_df = pd.concat([calls, puts], ignore_index=True)
    
    # Add date and expiry information
    options_df['Date'] = pd.to_datetime('today').normalize()
    
    # Parse expiry date from filename
    try:
        expiry_str = file_path.split('-')[-3:]
        expiry_date = ' '.join(expiry_str).replace('.csv', '')
        options_df['Expiry'] = pd.to_datetime(expiry_date, format='%d %b %Y')
    except:
        options_df['Expiry'] = pd.to_datetime('2025-08-14')  # Fallback date
    
    # Calculate time to expiration
    options_df['T'] = (options_df['Expiry'] - options_df['Date']).dt.days / 365.0
    
    # Convert all columns to appropriate types
    numeric_cols = ['OI', 'CHNG_IN_OI', 'VOLUME', 'IV', 'LTP', 'CHNG', 
                    'BID_QTY', 'BID', 'ASK', 'ASK_QTY', 'Strike Price']
    
    for col in numeric_cols:
        options_df[col] = (
            options_df[col]
            .astype(str)
            .str.replace(',', '')
            .replace(['-', ' ', 'nan', 'NaN', ''], '0')
            .astype(float)
        )
    
    # Filter valid options
    options_df = options_df[options_df['LTP'] > 0]
    
    return options_df

In [12]:
df

,Unnamed: 0,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID,ASK,...,BID.1,ASK.1,ASK QTY.1,CHNG.1,LTP.1,IV.1,VOLUME.1,CHNG IN OI.1,OI.1,Unnamed: 22
0,NaN,-,-,-,-,-,-,75,"2,188.45","2,461.10",...,52.55,54.45,75,0.65,52.55,15.89,159,105,317,NaN
1,NaN,-,-,-,-,-,-,75,"1,881.40","2,353.90",...,54.50,56.80,300,-,57.95,-,-,-,20,NaN
2,NaN,-,-,-,-,-,-,825,"1,819.40","2,362.40",...,57.30,59.50,300,0.85,58.75,15.79,19,9,43,NaN
3,NaN,-,-,-,-,-,-,825,"1,785.75","2,313.40",...,59.45,61.70,300,-5.85,61.55,15.70,5,5,30,NaN
4,NaN,1,-,-,-,"2,105.55",-,75,"2,094.20","2,265.40",...,62.85,64.90,600,1.90,64.95,15.64,74,-20,663,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,NaN,74,38,88,9.46,79.50,-7.90,300,79.00,81.00,...,"1,251.35","1,497.15",75,-,"1,509.80",-,-,-,3,NaN
75,NaN,-,-,-,-,-,-,-,-,145.00,...,"1,366.60","2,007.45","1,725",-,"1,553.40",-,-,-,2,NaN
76,NaN,195,5,163,9.57,69.50,-1.65,300,67.65,69.30,...,"1,410.80","1,591.55",75,-,"1,598.50",-,-,-,2,NaN
77,NaN,-,-,-,-,-,-,-,-,130.65,...,"1,450.05","1,639.60",75,-,"1,642.90",-,-,-,2,NaN
